#### Test att hämta bilder och söka i SPA
Och ladda ned till disk

* denna [Notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/Sveriges%20Riksdag%20i%20SPA.ipynb)
* se [massuppladda med UI](https://github.com/salgo60/Wikidata_riksdagen-corpus/issues/84#issuecomment-1399243509) --> att dubletter fångas och man kopplar Strukturerad data på bilden vid uppladdning

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-01-21 14:38:23.945995


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url= "https://portrattarkiv.se/endpoints/search.php"  
urlbasePic = "https://portrattarkiv.se/endpoints/file.php?id="

In [3]:
# Sök "riksdagar.)""
encoded_body = json.dumps({
        "limit": "1000",
        "from": "0",
        "facts":{},
        "factsExact":{},
        "all":"riksdagar.)",
    })
# All = Sveriges Riksdag 1924 --> 912
encoded_body = json.dumps({
        "limit": "10000",
        "from": "0",
        "facts":{},
        "factsExact":{},
        "all":"Sveriges Riksdag 1924"
    })

# PortraitCatalog = Sveriges Riksdag 1924 --> 912
PortraitCatalogSPA = "Sveriges Riksdag 1924"
encoded_body = json.dumps({
        "limit": "10000",
        "from": "0",
        "facts":{},
        "factsExact":{"PortraitCatalog":PortraitCatalogSPA},
#        "all":null,
    })

# PortraitCatalog = Riksdagsmän 1903 --> 241
PortraitCatalogSPA = "Riksdagsmän 1903"
encoded_body = json.dumps({
        "limit": "10000",
        "from": "0",
        "facts":{},
        "factsExact":{"PortraitCatalog":PortraitCatalogSPA},
#        "all":null,
    })



In [4]:
r = http.request('POST', url,
             headers={'Content-Type': 'application/json'},
             body=encoded_body)

if r.status != 200:
    print(r.status)
    

data = json.loads(r.data.decode('utf-8'),) 
urls = []

print("Number hits \t",len(data["hits"]["hits"]))


Number hits 	 241


In [5]:
for h in data["hits"]["hits"]:
    id = h["_id"]
    source = h["_source"]
    #print(id)
    try:
        urlPicture = urlbasePic + id
        print("\t",urlPicture)
        urls.append(urlPicture)
        #print(source)
        score = h["_score"]
        FirstName = source["FirstName"]
        LastName = source["LastName"]
        BirthYear = source["BirthYear"]
        BirthDate = source["BirthDate"]
        #wd = getWD(FirstName)
        #print("\t",score,FirstName, " ", LastName, " - ", BirthYear, "\t", )
    except:
        print("Error hdata")

	 https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAABNUKQ
	 https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAABNTzg
	 https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAABNTzA
	 https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAABNUKw
	 https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAABNToQ
	 https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAABNT2A
	 https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAABNUEg
	 https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAABNT0A
	 https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAABNUMw
	 https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAABNUEw
	 https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAABNUGw
	 https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAABNT4A
	 https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAABNT4Q
	 https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAA

In [6]:
from IPython.display import HTML, Image

def _src_from_data(data):
    """Base64 encodes image bytes for inclusion in an HTML img element"""
    img_obj = Image(data=data)
    for bundle in img_obj._repr_mimebundle_():
        for mimetype, b64value in bundle.items():
            if mimetype.startswith('image/'):
                return f'data:{mimetype};base64,{b64value}'

def gallery(images, row_height='auto'):
    """Shows a set of images in a gallery that flexes with the width of the notebook.
    
    Parameters
    ----------
    images: list of str or bytes
        URLs or bytes of images to display

    row_height: str
        CSS height value to assign to all images. Set to 'auto' by default to show images
        with their native dimensions. Set to a value like '250px' to make all rows
        in the gallery equal height.
    """
    figures = []
    for image in images:
        if isinstance(image, bytes):
            src = _src_from_data(image)
            caption = ''
        else:
            src = image
            caption = f'<figcaption style="font-size: 0.6em">{image}</figcaption>'
        figures.append(f'''
            <figure style="margin: 5px !important;">
              <img src="{src}" style="height: {row_height}">
              {caption}
            </figure>
        ''')
    return HTML(data=f'''
        <div style="display: flex; flex-flow: row wrap; text-align: center;">
        {''.join(figures)}
        </div>
    ''')

In [7]:
for h in data["hits"]["hits"]:
    id = h["_id"]
    source = h["_source"]
    urlPicture = urlbasePic + id
    urls.append(urlPicture)
gallery(urls, row_height='100px')

In [8]:
import requests
#Hämtar bild katalog
for h in data["hits"]["hits"]:
    id = h["_id"]
    source = h["_source"]
    urlPicture = urlbasePic + id
    urls.append(urlPicture)
    filename = "./temp/" + id + ".jpg"
    print (filename)
    response = requests.get(urlPicture)
    file = open(filename, "wb")
    file.write(response.content)
    file.close()

./temp/sj9PGLAlnmUAAAAAABNUKQ.jpg
./temp/sj9PGLAlnmUAAAAAABNTzg.jpg
./temp/sj9PGLAlnmUAAAAAABNTzA.jpg
./temp/sj9PGLAlnmUAAAAAABNUKw.jpg
./temp/sj9PGLAlnmUAAAAAABNToQ.jpg
./temp/sj9PGLAlnmUAAAAAABNT2A.jpg
./temp/sj9PGLAlnmUAAAAAABNUEg.jpg
./temp/sj9PGLAlnmUAAAAAABNT0A.jpg
./temp/sj9PGLAlnmUAAAAAABNUMw.jpg
./temp/sj9PGLAlnmUAAAAAABNUEw.jpg
./temp/sj9PGLAlnmUAAAAAABNUGw.jpg
./temp/sj9PGLAlnmUAAAAAABNT4A.jpg
./temp/sj9PGLAlnmUAAAAAABNT4Q.jpg
./temp/sj9PGLAlnmUAAAAAABNUXw.jpg
./temp/sj9PGLAlnmUAAAAAABNTXg.jpg
./temp/sj9PGLAlnmUAAAAAABNUIw.jpg
./temp/sj9PGLAlnmUAAAAAABNT_w.jpg
./temp/sj9PGLAlnmUAAAAAABNTpQ.jpg
./temp/sj9PGLAlnmUAAAAAABNURg.jpg
./temp/sj9PGLAlnmUAAAAAABNTbQ.jpg
./temp/sj9PGLAlnmUAAAAAABNTkQ.jpg
./temp/sj9PGLAlnmUAAAAAABNT2Q.jpg
./temp/sj9PGLAlnmUAAAAAABNT4g.jpg
./temp/sj9PGLAlnmUAAAAAABNUUg.jpg
./temp/sj9PGLAlnmUAAAAAABNULg.jpg
./temp/sj9PGLAlnmUAAAAAABNUDQ.jpg
./temp/sj9PGLAlnmUAAAAAABNTiQ.jpg
./temp/sj9PGLAlnmUAAAAAABNTlA.jpg
./temp/sj9PGLAlnmUAAAAAABNULw.jpg
./temp/sj9PGLA

In [9]:
# Create csv file
# Match it in Open Refine 

header = ['id', 'urlPicture', 'FirstName','LastName','BirthYear','PortraitCatalog']
list_pm = []
PortraitCatalog = ""
for h in data["hits"]["hits"]:
    id = h["_id"]
    source = h["_source"]
    #print(source)
    urlPicture = urlbasePic + id
    FirstName = source["FirstName"]
    LastName = source["LastName"]
    BirthYear = source["BirthYear"]
    BirthDate = source["BirthDate"]
    list_pm.append([id,urlPicture,FirstName,LastName,BirthDate,PortraitCatalogSPA])

In [10]:
#list_pm 
list_pm_df = pd.DataFrame(list_pm)
list_pm_df.columns = header

In [11]:
list_pm_df

,id,urlPicture,FirstName,LastName,BirthYear,PortraitCatalog
0,sj9PGLAlnmUAAAAAABNUKQ,https://portrattarkiv.se/endpoints/file.php?id...,Henrik,Fredholm,1838-07-08,Riksdagsmän 1903
1,sj9PGLAlnmUAAAAAABNTzg,https://portrattarkiv.se/endpoints/file.php?id...,Knut,von Geijer,1864-05-12,Riksdagsmän 1903
2,sj9PGLAlnmUAAAAAABNTzA,https://portrattarkiv.se/endpoints/file.php?id...,Johannes,Bengtsson,1844-10-27,Riksdagsmän 1903
3,sj9PGLAlnmUAAAAAABNUKw,https://portrattarkiv.se/endpoints/file.php?id...,August,Henricson,None,Riksdagsmän 1903
4,sj9PGLAlnmUAAAAAABNToQ,https://portrattarkiv.se/endpoints/file.php?id...,David,Holmgren,None,Riksdagsmän 1903
...,...,...,...,...,...,...
236,sj9PGLAlnmUAAAAAABNUFw,https://portrattarkiv.se/endpoints/file.php?id...,Fredrik,Barnekow,None,Riksdagsmän 1903
237,sj9PGLAlnmUAAAAAABNUXQ,https://portrattarkiv.se/endpoints/file.php?id...,Ossian,Berger,1849-02-13,Riksdagsmän 1903
238,sj9PGLAlnmUAAAAAABNT1Q,https://portrattarkiv.se/endpoints/file.php?id...,Aron,Gunnarsson,1855-10-25,Riksdagsmän 1903
239,sj9PGLAlnmUAAAAAABNT2w,https://portrattarkiv.se/endpoints/file.php?id...,Gustaf,Österberg,1864-09-18,Riksdagsmän 1903


In [12]:
csvFile = "OpenRefine_" + PortraitCatalogSPA + ".csv"
list_pm_df.to_csv(csvFile)

## Tvåkammar-riksdagen 1867-

In [13]:
PortraitCatalogSPA = "Tvåkammar-riksdagen 1867-"
encoded_body = json.dumps({
        "limit": "10000",
        "from": "0",
        "facts":{},
        "factsExact":{"PortraitCatalog":PortraitCatalogSPA},
#        "all":null,
    })

In [14]:
r = http.request('POST', url,
             headers={'Content-Type': 'application/json'},
             body=encoded_body)

if r.status != 200:
    print(r.status)
    

data = json.loads(r.data.decode('utf-8'),) 
urls = []

print("Number hits \t",len(data["hits"]["hits"]))

Number hits 	 4353


In [15]:
for h in data["hits"]["hits"]:
    id = h["_id"]
    source = h["_source"]
    #print(id)
    try:
        urlPicture = urlbasePic + id
        #print("\t",urlPicture)
        urls.append(urlPicture)
        #print(source)
        score = h["_score"]
        FirstName = source["FirstName"]
        LastName = source["LastName"]
        BirthYear = source["BirthYear"]
        BirthDate = source["BirthDate"]
        #wd = getWD(FirstName)
        #print("\t",score,FirstName, " ", LastName, " - ", BirthYear, "\t", )
    except:
        print("Error hdata")

In [16]:
for h in data["hits"]["hits"]:
    id = h["_id"]
    source = h["_source"]
    urlPicture = urlbasePic + id
    urls.append(urlPicture)
gallery(urls, row_height='100px')

In [17]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2023-01-21 14:40:44.969937
Time elapsed (hh:mm:ss.ms) 0:02:21.024788
